The master_file serves as the glue for all other files in the project.  It is the central file to load training and test data, tag the addresses, standardize the addresses, and compare the different address lists.

In [4]:
from collections import defaultdict
from address_compare import reference_data as refdt
from address_compare import standardizers as stndrdzr
from address_compare import comparers as comps
from address_compare import matcher as mtch
import json
import pandas as pd

In [5]:
retrain_crf_tagger = False

In [6]:
# Placeholder for reading/calling the training data for the CRF Tagger and sending the training data to train the model
if retrain_crf_tagger:
    with open('data/tagged_addresses.json') as f:
        td = json.load(f)
    
    #send training data to CRF tagger to train the model here...

In [7]:
# Placeholder for reading/calling the 2 lists of raw addresses
raw_address_list_1 = None
raw_address_list_2 = None

In [9]:
# Placeholder for calling the trained CRF Tagger on the 2 lists of raw addresses